In [18]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense,
    Concatenate,
    Input,
    Lambda,
    GlobalAveragePooling2D,
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [19]:
user_df = pd.read_csv("../dataset/users.csv", delimiter=";")
artwork_df = pd.read_csv("../dataset/artworks.csv", delimiter=";")
artwork_df.fillna(-1, inplace=True)
user_df.fillna(-1, inplace=True)
artworks = artwork_df.drop_duplicates(subset="title", keep="first")
users = user_df.drop_duplicates(subset="name", keep="first")
artwork_df["tag_string"] = artwork_df["image_tags"].apply(
    lambda x: " ".join(ast.literal_eval(x))
)

In [20]:
tfidf_vectorizer = TfidfVectorizer()
tag_vectors = tfidf_vectorizer.fit_transform(artwork_df["tag_string"])

In [21]:
def get_similar_image(user_artworks, artwork_df, tag_vectors):
    user_tags_string = " ".join(user_artworks["tag_string"].values)
    user_tags_vector = tfidf_vectorizer.transform([user_tags_string])
    similarity = cosine_similarity(user_tags_vector, tag_vectors)
    average_similarity = similarity.mean(axis=0)
    similar_image_index = average_similarity.argmax()
    return pd.DataFrame([artwork_df.iloc[similar_image_index]])

In [22]:
def preprocess_image(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image = image.convert("RGB")
        image = image.resize((224, 224))
        image_array = np.array(image)
        image_array = image_array / 255.0
        return image_array
    else:
        raise FileNotFoundError(
            f"Impossibile scaricare l'immagine dall'URL: {image_url}"
        )

In [23]:
def create_triples(user_df, artwork_df):
    triples = []
    for _, user_row in user_df.iterrows():
        user_interactions = artwork_df[artwork_df["author"] == user_row["name"]]
        Pu = user_interactions.sample(frac=1)
        i = get_similar_image(user_interactions, artwork_df, tag_vectors)
        non_interacted = artwork_df[~artwork_df["author"].isin([user_row["name"]])]
        j = non_interacted.sample()
        triples.append((Pu, i, j))
    return triples

In [24]:
def preprocess_data(triples, image_preprocessor):
    processed_triples = []

    for Pu, i, j in triples:
        Pu_images = np.array([image_preprocessor(url) for url in Pu["img"].values])
        i_image = np.array(image_preprocessor(i["img"].values[0]))
        j_image = np.array(image_preprocessor(j["img"].values[0]))
        processed_triples.append(
            (Pu_images, i_image, j_image,i.iloc[0])
        )

    return processed_triples

In [25]:
embedding_dim = 200
pu_dim = 400
margin = 0.4

In [26]:
resnet = ResNet50(weights="imagenet", include_top=False, pooling=None)
for layer in resnet.layers:
    layer.trainable = False


def extract_features(image):
    features = resnet(image)
    features = GlobalAveragePooling2D()(features)
    return features

In [27]:
def custom_reduce_sum(x, y):
    return K.sum(x * y, axis=1)

In [28]:
def custom_triplet_loss(y_true, y_pred, margin=0.4):
    score_i, score_j = tf.split(y_pred, num_or_size_splits=2, axis=-1)
    loss = tf.maximum(0.0, margin + score_j - score_i)
    return tf.reduce_mean(loss)

In [29]:
input_pu = Input(shape=(3, 224, 224, 3), name="input_pu")
input_i = Input(shape=(224, 224, 3), name="input_i")
input_j = Input(shape=(224, 224, 3), name="input_j")
pu_features = Lambda(
    lambda x: K.map_fn(lambda y: extract_features(y), x),
    output_shape=(None, embedding_dim),
)(input_pu)

i_features = extract_features(input_i)
j_features = extract_features(input_j)

dense_layer_1 = Dense(embedding_dim, activation="selu", name="dense_layer_1")
dense_layer_2 = Dense(embedding_dim, activation="selu", name="dense_layer_2")

reduced_pu = Lambda(
    lambda x: K.map_fn(lambda y: dense_layer_2(dense_layer_1(y)), x),
    output_shape=(None, embedding_dim),
)(pu_features)


reduced_i = dense_layer_2(dense_layer_1(i_features))
reduced_j = dense_layer_2(dense_layer_1(j_features))


average_pooled_pu = Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)
max_pooled_pu = Lambda(lambda x: K.max(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)

pooled_pu = Concatenate()([average_pooled_pu, max_pooled_pu])


pu_dense_1 = Dense(300, activation="selu", name="pu_dense_1")(pooled_pu)
pu_dense_2 = Dense(200, activation="selu", name="pu_dense_2")(pu_dense_1)

final_pu = Dense(200, activation="selu", name="pu_dense_3")(pu_dense_2)

In [30]:
score_i = Lambda(lambda x: K.sum(x[0] * x[1], axis=1, keepdims=True))(
    [final_pu, reduced_i]
)
score_j = Lambda(lambda x: K.sum(x[0] * x[1], axis=1, keepdims=True))(
    [final_pu, reduced_j]
)
output_scores = Concatenate(axis=-1)([score_i, score_j])

In [31]:
def build_curatornet_model():
    tf.keras.backend.clear_session()
    curatornet = Model(
        inputs=[
            input_pu,
            input_i,
            input_j,
        ],
        outputs=output_scores,
    )
    return curatornet

# curatorenet_model = build_curatornet_model()
# curatorenet_model.compile(optimizer="adam", loss=custom_triplet_loss)
# curatorenet_model.summary()

In [32]:
def prepare_inputs(
    processed_triples, users_df, target_image_count=3, expected_length=183
):
    inputs = {
        "input_pu": [],
        "input_i": [],
        "input_j": [],
    }

    for pu_images, i_image, j_image, i_meta in processed_triples:
        user_features = users_df[users_df["name"] == i_meta["author"]].iloc[0]
        if pu_images.shape[0] < target_image_count:
            pad_width = target_image_count - pu_images.shape[0]
            pu_images = np.pad(
                pu_images, ((0, pad_width), (0, 0), (0, 0), (0, 0)), mode="constant"
            )
        elif pu_images.shape[0] > target_image_count:
            pu_images = pu_images[:target_image_count]

        inputs["input_pu"].append(pu_images)
        inputs["input_i"].append(i_image)
        inputs["input_j"].append(j_image)

    for key in inputs:
        try:
            if not isinstance(inputs[key], np.ndarray):
                inputs[key] = np.array(inputs[key])

            current_length = len(inputs[key])
            if current_length != expected_length:
                print(f"Adjusting {key} from {current_length} to {expected_length}")
                repeat_factor = max(1, expected_length // current_length)
                if repeat_factor > 1:
                    inputs[key] = np.tile(
                        inputs[key], (repeat_factor,) + (1,) * (inputs[key].ndim - 1)
                    )
                if len(inputs[key]) > expected_length:
                    inputs[key] = inputs[key][:expected_length]
        except Exception as e:
            print(f"Warning: Could not convert {key} to numpy array: {e}")
    return inputs

In [33]:
def convert_inputs_to_tensors(inputs):
    for key, value in inputs.items():
        if isinstance(value, np.ndarray) or isinstance(value, list):
            inputs[key] = tf.convert_to_tensor(value)
    return inputs

In [34]:
triples = create_triples(user_df, artwork_df)
preprocessed_triples = preprocess_data(triples, preprocess_image)
filtered_triples = [triple for triple in preprocessed_triples if triple[0].ndim == 4]
inputs = prepare_inputs(filtered_triples, user_df)
convert_inputs_to_tensors(inputs)

Adjusting input_pu from 964 to 183
Adjusting input_i from 964 to 183
Adjusting input_j from 964 to 183


{'input_pu': <tf.Tensor: shape=(183, 3, 224, 224, 3), dtype=float64, numpy=
 array([[[[[0.05882353, 0.08235294, 0.12941176],
           [0.0627451 , 0.08627451, 0.13333333],
           [0.06666667, 0.09411765, 0.1372549 ],
           ...,
           [0.02745098, 0.12941176, 0.22352941],
           [0.02745098, 0.12941176, 0.21960784],
           [0.02745098, 0.12941176, 0.21960784]],
 
          [[0.0627451 , 0.08627451, 0.13333333],
           [0.06666667, 0.09019608, 0.1372549 ],
           [0.06666667, 0.09411765, 0.1372549 ],
           ...,
           [0.02352941, 0.10588235, 0.18823529],
           [0.02352941, 0.10588235, 0.18431373],
           [0.02352941, 0.10588235, 0.18431373]],
 
          [[0.0627451 , 0.08627451, 0.13333333],
           [0.06666667, 0.09019608, 0.1372549 ],
           [0.07058824, 0.09803922, 0.14117647],
           ...,
           [0.02352941, 0.08627451, 0.14117647],
           [0.02352941, 0.08627451, 0.14117647],
           [0.02352941, 0.08627451, 0

In [35]:
X_train, X_test = {}, {}
for key, value in inputs.items():
    print(f"Splitting {key}")
    indices = tf.range(start=0, limit=tf.shape(value)[0], dtype=tf.int32)
    train_idx, test_idx = train_test_split(
        indices.numpy(), test_size=0.2, random_state=42
    )
    train_idx = tf.convert_to_tensor(train_idx, dtype=tf.int32)
    test_idx = tf.convert_to_tensor(test_idx, dtype=tf.int32)
    X_train[key] = tf.gather(value, train_idx)
    X_test[key] = tf.gather(value, test_idx)
    print(
        f"Key: {key}, dtype: {value.dtype}, train shape: {X_train[key].shape}, test shape: {X_test[key].shape}"
    )

Splitting input_pu
Key: input_pu, dtype: <dtype: 'float64'>, train shape: (146, 3, 224, 224, 3), test shape: (37, 3, 224, 224, 3)
Splitting input_i
Key: input_i, dtype: <dtype: 'float64'>, train shape: (146, 224, 224, 3), test shape: (37, 224, 224, 3)
Splitting input_j
Key: input_j, dtype: <dtype: 'float64'>, train shape: (146, 224, 224, 3), test shape: (37, 224, 224, 3)


In [37]:
labels = np.zeros((len(triples), 1))
train_labels = labels[:146]
test_labels = labels[146 : 146 + 37]
curatornet_model = build_curatornet_model()
curatornet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
    loss=custom_triplet_loss,
    metrics=["accuracy"],
)
curatornet_model.fit(X_train, train_labels, epochs=10, batch_size=32)
val_loss, val_acc = curatornet_model.evaluate(X_test, test_labels)
print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")

Epoch 1/10
Instructions for updating:
Use fn_output_signature instead
5/5 ━━━━━━━━━━━━━━━━━━━━ 48s 5s/step - accuracy: 0.5743 - loss: 1.2407
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 5s/step - accuracy: 0.5487 - loss: 1.1167
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.5606 - loss: 1.1949
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 5s/step - accuracy: 0.5575 - loss: 1.0682
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 5s/step - accuracy: 0.5086 - loss: 1.1547
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.5763 - loss: 1.0721
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.5473 - loss: 1.0957
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.5767 - loss: 0.9103
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.5630 - loss: 0.9180
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.5660 - loss: 0.8772


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 37, 37, 37
'y' sizes: 824


In [39]:
test_labels = labels[146 : 146 + 37]
val_loss, val_acc = curatornet_model.evaluate(X_test, test_labels)
print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 979ms/step - accuracy: 0.5374 - loss: 0.9918
Validation loss: 0.9730175137519836, Validation accuracy: 0.5405405163764954


In [45]:
embedding_model = Model(
    inputs=curatornet_model.input,
    outputs=[
        curatornet_model.get_layer("pu_dense_3").output,
        curatornet_model.get_layer("dense_layer_2").output,
    ],
)
user_embeddings, art_embeddings = embedding_model.predict(inputs)

6/6 ━━━━━━━━━━━━━━━━━━━━ 36s 6s/step


In [46]:
def precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    return len(set(predicted_at_k) & actual_set) / k


def recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    return len(set(predicted_at_k) & actual_set) / len(actual_set)


def ndcg_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    dcg = sum(
        [
            1.0 / np.log2(i + 2) if predicted_at_k[i] in actual_set else 0.0
            for i in range(k)
        ]
    )
    idcg = sum([1.0 / np.log2(i + 2) for i in range(min(len(actual), k))])
    return dcg / idcg if idcg > 0 else 0.0

In [47]:
def get_top_k_recommendations(user_embedding, artwork_embeddings, k=10):
    similarities = np.dot(artwork_embeddings, user_embedding)
    top_k_indices = np.argsort(similarities)[::-1][:k]
    return top_k_indices

In [48]:
def top_k_artworks_by_popularity_score(k):
    cleaned_artworks_df = artwork_df.sort_values(
        "number_of_views", ascending=False
    ).drop_duplicates(subset=["title", "author"], keep="first")
    cleaned_artworks_df["popularity_score"] = (
        cleaned_artworks_df["number_of_views"] * 0.2
        + cleaned_artworks_df["likes"] * 0.5
        + cleaned_artworks_df["number_of_comments"] * 0.3
    )
    top_k_artworks = cleaned_artworks_df.nlargest(k, "popularity_score")
    return top_k_artworks.index.tolist()

In [49]:
test = get_top_k_recommendations(user_embeddings[0], art_embeddings, k=200)
views_set = top_k_artworks_by_popularity_score(100)
precision = precision_at_k(views_set, test, 50)
recall = recall_at_k(views_set, test, 50)
ndcg = ndcg_at_k(views_set, test, 50)
print(f"Precision: {precision}, Recall: {recall}, NDCG: {ndcg}")

Precision: 0.12666666666666668, Recall: 0.095, NDCG: 0.12345585215434403
